In [1]:
from src.features import preprocessing
import river

In [3]:
DATA_PATH = "./datasets/traffic/"

list_of_datafiles = preprocessing.get_list_of_datafiles(DATA_PATH, sort = True)
df = preprocessing.load_data(DATA_PATH, list_of_datafiles)
train_df, test_df = preprocessing.make_train_test_datasets(df, split_point=20000)

100%|██████████| 30000/30000 [00:58<00:00, 513.23it/s] 


In [27]:
n_input_steps = 1
n_output_steps = 1
#target = '5->8' # SMAPE: ~3.5
#target = '8->5' # SMAPE: ~3.5
#target = '5->12' # SMAPE: ~20.0
target = '8->12' # SMAPE: ~7.5
X, y = preprocessing.split_sequence(sequence = df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)
#X_test, y_test = preprocessing.split_sequence(sequence = test_df[target], n_input_steps=n_input_steps, n_output_steps=n_output_steps)

100%|██████████| 30000/30000 [00:02<00:00, 12311.98it/s]


In [28]:
# y_t = [y[0] for y in y_train]

In [29]:
X[1]

array([200.265])

In [30]:
y[1]

array([200.398])

In [31]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
# n_features = 1
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

In [32]:
import numpy as np

In [33]:
# model = river.compose.Pipeline(
#     river.preprocessing.StandardScaler(),
#     river.linear_model.LinearRegression(river.optim.SGD(lr=0.3))
#     )
# metric = river.metrics.SMAPE()

# predicted = list()
# for xi, yi in river.stream.iter_array(X, y):
#     # Test the current model on the new "unobserved" sample
#     yi_pred = model.predict_one(xi)

#     # Update the running metric with the prediction and ground truth value
#     metric.update(yi[0], yi_pred)
#     predicted.append(yi_pred)

#     # Train the model with the new sample
#     model.learn_one(xi, yi[0])


# print(f'ROC AUC: {metric}')

In [34]:
models = [
river.linear_model.LinearRegression(optimizer=river.optim.SGD(lr=lr))
for lr in [0.0001, 0.001, 1e-05, 0.01]
]

model = (
river.preprocessing.StandardScaler() |
    river.model_selection.EpsilonGreedyRegressor(
    models,
    epsilon=0.1,
    decay=0.001,
    burn_in=100,
    seed=1
    )
)

metric = river.metrics.SMAPE()

predicted = list()
for xi, yi in river.stream.iter_array(X, y):
    # Test the current model on the new "unobserved" sample
    yi_pred = model.predict_one(xi)

    # Update the running metric with the prediction and ground truth value
    metric.update(yi[0], yi_pred)
    predicted.append(yi_pred)

    # Train the model with the new sample
    model.learn_one(xi, yi[0])


print(f'ROC AUC: {metric}')
model['EpsilonGreedyRegressor'].best_model

ROC AUC: SMAPE: 7.476457


LinearRegression (
  optimizer=SGD (
    lr=Constant (
      learning_rate=0.01
    )
  )
  loss=Squared ()
  l2=0.
  intercept_init=0.
  intercept_lr=Constant (
    learning_rate=0.01
  )
  clip_gradient=1e+12
  initializer=Zeros ()
)

In [35]:
for i in range(len(predicted)):
    print(f"real: {y[i]} | pred: {predicted[i]}")

real: [200.265] | pred: 0.0
real: [200.398] | pred: 4.0053
real: [200.529] | pred: 7.981200020428078
real: [200.658] | pred: 23.07637793546847
real: [200.786] | pred: 33.875470838723835
real: [200.912] | pred: 44.64916379075169
real: [201.037] | pred: 55.088028156501835
real: [201.159] | pred: 65.107331156302
real: [201.28] | pred: 74.51597179538703
real: [201.4] | pred: 83.42697768186312
real: [201.518] | pred: 91.82459980088322
real: [201.634] | pred: 99.639283335977
real: [201.748] | pred: 106.89679977135827
real: [201.861] | pred: 113.62672947685755
real: [201.972] | pred: 119.93325773088839
real: [202.082] | pred: 125.75602583985855
real: [202.19] | pred: 131.2037788379271
real: [202.296] | pred: 136.21685298546066
real: [202.401] | pred: 140.82970857640183
real: [202.504] | pred: 145.14842444809534
real: [202.605] | pred: 149.11069832281862
real: [202.704] | pred: 152.74669139368154
real: [202.802] | pred: 156.08375653278242
real: [202.898] | pred: 159.21963551614064
real: [202.9